In [2]:
import preprocessing_util as util
import os
import numpy as np
import pandas as pd
import torch

In [ ]:
def preprocess(data_dir, dest_dir, max_dimensions=[400,400,400], resample_resolution=[2,2,2]):
    train_images_dir = os.path.join(data_dir, 'train_images/')
    train_label_coordinates_dir = os.path.join(data_dir, 'train_label_coordinates.csv')
    train_series_descriptions_dir = os.path.join(data_dir, 'train_series_descriptions.csv')
    train_labels_dir = os.path.join(data_dir, 'train.csv')
    
    studies = os.listdir(train_images_dir)
    
    for study in studies:
        series_ids = os.listdir(os.path.join(train_images_dir, study))
        
        # Get the Severity Label data for this study (one-hot encoded)
        labels = util.load_label_data(train_labels_dir, study)
        
        for series in series_ids:
            
            scan = util.load_sample(os.path.join(train_images_dir, study, series))
            
            # Skip this series if it has an error
            if not util.check_valid_scan(scan):
                print(f'Skipping study {study} series {series}, multiple scans dumped in same directory')
                continue
            
            # Get the coordinate data for this series
            coord_data, mask = util.load_coord_data(train_label_coordinates_dir, series)
            
            # Stack the scan together
            slices = np.stack([s.pixel_array for s in scan], dtype=np.float32)
            
            # Resample the Images and Coordinates to new resolution
            resampled_image, resampled_coord_data, new_spacing = util.resample_with_coordinates( slices , scan, coord_data, resample_resolution)
            
            orientation = util.get_series_orientation(train_series_descriptions_dir, series)
            if orientation.lower().contains('sagittal'):
                resampled_image, resampled_coord_data = util.sagittal_to_axial(resampled_image, resampled_coord_data)
                
            
            
            

            
        
    
    return